In [4]:
import cv2
import numpy as np
import mediapipe as mp
import screen_brightness_control as sbc
from math import hypot, sqrt
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from collections import deque


def main():
    # Audio setup for controlling system volume
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    volRange = volume.GetVolumeRange()
    minVol, maxVol, _ = volRange

    # Initialize MediaPipe Hand Detection
    mpHands = mp.solutions.hands
    hands = mpHands.Hands(
        static_image_mode=False,
        model_complexity=1,
        min_detection_confidence=0.8,
        min_tracking_confidence=0.8,
        max_num_hands=2)

    draw = mp.solutions.drawing_utils
    cap = cv2.VideoCapture(0)
    
    
    if not cap.isOpened():
        print("Error: Could not open video capture")
        return

    # Control variables
    smooth_brightness = sbc.get_brightness()[0]
    smooth_volume = volume.GetMasterVolumeLevelScalar() * 100
    prev_volume = smooth_volume
    
    # Parameters for control
    MIN_DISTANCE = 20
    MAX_DISTANCE = 230
    SMOOTHING_FACTOR = 0.1  # Reduced smoothing factor for quicker response
    
    # Smoothing queues
    brightness_queue = deque(maxlen=5)  # Adjust queue length for desired smoothness
    volume_queue = deque(maxlen=5)

    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)  # Mirror the frame
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            processed = hands.process(frameRGB)

            # Initialize landmark lists
            left_landmark_list = []
            right_landmark_list = []

            if processed.multi_hand_landmarks:
                for i, hand_landmarks in enumerate(processed.multi_hand_landmarks):
                    # Get handedness (left/right) - this is the key fix
                    if processed.multi_handedness:
                        hand_label = processed.multi_handedness[i].classification[0].label
                    else:
                        # Default to right hand if handedness isn't available
                        hand_label = "Right" if i == 0 else "Left"
                    
                    # Get landmarks in pixel coordinates
                    landmarks = [(landmark.x, landmark.y) for landmark in hand_landmarks.landmark]
                    
                    # Assign to correct list based on hand label
                    if hand_label == "Left":
                        left_landmark_list = landmarks
                    else:
                        right_landmark_list = landmarks
                    
                    # Draw landmarks
                    draw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)

            # Brightness control with left hand
            if left_landmark_list:
                left_distance = get_distance(left_landmark_list, frame)
                if left_distance > MIN_DISTANCE:
                    b_level = np.interp(left_distance, [MIN_DISTANCE, MAX_DISTANCE], [0, 100])
                    brightness_queue.append(b_level)
                    smooth_brightness = np.mean(brightness_queue) #Simple average smoothing
                    sbc.set_brightness(int(np.clip(smooth_brightness, 0, 100)))
                else:
                    sbc.set_brightness(0) # Ensure minimum brightness when hand is close

            # Volume control with right hand - THIS IS NOW FIXED
            if right_landmark_list:
                right_distance = get_distance(right_landmark_list, frame)
                if right_distance > MIN_DISTANCE:
                    vol_level = np.interp(right_distance, [MIN_DISTANCE, MAX_DISTANCE], [0, 1])
                    volume_queue.append(vol_level * 100)
                    smooth_volume = np.mean(volume_queue)
                    volume.SetMasterVolumeLevelScalar(np.clip(smooth_volume / 100, 0, 1), None)
                    prev_volume = smooth_volume
                else:
                    volume.SetMasterVolumeLevelScalar(0, None) # Mute when hand is close
                    prev_volume = 0

            # Display information
            cv2.putText(frame, f"Brightness: {int(smooth_brightness)}%", (10, 30),  
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, f"Volume: {int(prev_volume)}%", (10, 60),  
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Show which hands are detected
            cv2.putText(frame, f"Left Hand: {'Yes' if left_landmark_list else 'No'}", (10, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, f"Right Hand: {'Yes' if right_landmark_list else 'No'}", (10, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            cv2.imshow('GestureTune', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
    except Exception as e:
        print(f"Error: {e}")
    finally:
        cap.release()
        cv2.destroyAllWindows()

def get_distance(landmark_list, frame):
    """Calculate distance between thumb and index finger tips"""
    if len(landmark_list) < 9:
        return 0

    # Thumb tip (4) and index finger tip (8)
    x1, y1 = int(landmark_list[4][0] * frame.shape[1]), int(landmark_list[4][1] * frame.shape[0])
    x2, y2 = int(landmark_list[8][0] * frame.shape[1]), int(landmark_list[8][1] * frame.shape[0])

    # Draw visual feedback
    cv2.circle(frame, (x1, y1), 7, (0, 255, 0), cv2.FILLED)
    cv2.circle(frame, (x2, y2), 7, (0, 255, 0), cv2.FILLED)
    cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
    cv2.putText(frame, "Left=Brightness | Right=Volume", (10, frame.shape[0]-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,255), 1)

    return hypot(x2 - x1, y2 - y1)

if __name__ == '__main__':
    main()